# TMA Analysis code supporting technote SITCOMTN-112
Craig Lage - 19-Feb-24

This notebook attempts to answer the following questions associated with the TMA: 

(1) What is the tracking error at the InPosition timestamp?\
(2) How long does the TMA take to settle after the InPosition timestamp?\
(3) What is the RMS tracking jitter for a 15 second period after the InPosition timestamp?\
(4) What is the time from the beginning of a slew until the TMA is InPosition and settled?



# Note that this must use the summit_utils tickets/DM-42039 branch until this branch is merged.

# Prepare the notebook

In [1]:
# Directory to store the data
from pathlib import Path
dataDir = Path("./plots")
dataDir.mkdir(exist_ok=True, parents=True)

# You can include a list of different days and blocks,
# including more than one block on the same day, if desired
# You can also create a loop to build this list.
#dayBlockPairs = [[20231214, 146], [20231215, 146]]
#dayBlockPairs = [[20231220, 146], [20231221, 146]]
#dayBlockPairs = [[20240212, 190], [20240212, 225]]
dayBlockPairs = [[20240208, 227]]


In [2]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline
from scipy.optimize import minimize
from lsst_efd_client import EfdClient
from lsst.summit.utils import getCurrentDayObs_int, dayObsIntToString
from lsst.summit.utils.tmaUtils import TMAEventMaker, plotEvent, getAzimuthElevationDataForEvent
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import getEfdData

In [3]:
client = EfdClient("usdf_efd")
eventMaker = TMAEventMaker()

In [ ]:
def getPreviousSlew(events, seqNum):
    # Find the previous slew associated with an event
    for event in events:
        if event.seqNum == seqNum - 1:
            return event

def filterBadValues(values, maxDelta=0.05):
    # Find non-physical points and replace with extrapolation 
    # This is in summit_utils, but is reproduced here to allow changes
    badCounter = 0
    consecutiveCounter = 0
    lastIndexReplaced = 0
    for i in range(2, len(values)):
        if consecutiveCounter > 3:
            #print("Hitting consecutive counter limit")
            consecutiveCounter = 0
            continue
        if abs(values[i] - values[i-1]) > maxDelta:
            #print(f"Index {i} is bad!")
            # This is a bogus value - replace it with an extrapolation                                                            
            # Of the preceding two values
            if i == lastIndexReplaced + 1:
                consecutiveCounter += 1
            else:
                consecutiveCounter = 0
            values[i] = (2.0 * values[i-1] - values[i-2])
            badCounter += 1
            lastIndexReplaced = i
    return badCounter

def jitterNextFifteen(track, azimuthData, elevationData, thisEl, delay):
    start = track.begin + TimeDelta(delay, format='sec')
    end = start + TimeDelta(15.0, format='sec')
    thisAzData = azimuthData.loc[start.isot:end.isot]
    thisAzError = thisAzData['azError']
    thisElData = elevationData.loc[start.isot:end.isot]
    thisElError = thisElData['elError']
    thisAzRms = np.sqrt(np.mean(thisAzError**2))
    thisElRms = np.sqrt(np.mean(thisElError**2))
    rmsError = np.sqrt(thisElRms**2 
                   + (thisAzRms * np.cos(thisEl * np.pi / 180.0))**2)
    return rmsError

def binary_search(track, azimuthData, elevationData, thisEl, 
                  delay_start, delay_end, precision=0.1):

        
    if jitterNextFifteen(track, azimuthData, elevationData, 
                         thisEl, delay_start) < 0.01:
        return(delay_start)
    if jitterNextFifteen(track, azimuthData, elevationData, 
                         thisEl, delay_end) > 0.01:
        return(delay_end)
    while (delay_end - delay_start) > precision:
        delay_mid = (delay_start + delay_end) / 2
        rmsError = jitterNextFifteen(track, azimuthData, 
                                     elevationData, thisEl, delay_mid)
        if rmsError < 0.01:
            delay_end = delay_mid
            last_good = delay_mid
        else:
            delay_start = delay_mid
            delay_end = last_good
    rmsError = jitterNextFifteen(track, azimuthData, 
                                     elevationData, thisEl, last_good)
    return delay_start



# Make a tracking plot for a single event.

In [ ]:
%matplotlib inline
dayObs = 20231220
seqNum = 431
event = eventMaker.getEvent(dayObs, seqNum)
fig = plt.figure(figsize=(10, 8))
plotEvent(client, event, fig, doFilterResiduals=True)
plt.savefig(str(dataDir / f"RubinTV_Tracking_{dayObs}-{seqNum}.png"))

# Now cycle through many events plotting the tracking errors, and getting the data for the slew time histogram and the jitter error histograms.

In [ ]:
# These are for plotting the slew time distributions
slewTimesInPosition = []
slewTimesSettled = []
slewDistance = []
azVals = []
elVals = []
shifts = []

rmsError0s = [] # This is the error at the inPosition

# These are for plotting the jitter distributions
#We do this for several delays after the InPosition
delays = [0.0, 1.0, 2.0, 3.0]
rmsErrors = {}
for delay in delays:
    rmsErrors[delay] = []

timeToSettles = []

firstDayObs = dayBlockPairs[0][0]
lastDayObs = dayBlockPairs[-1][0]

pdf = PdfPages(str(dataDir / f"Mount_Jitter_Plots_MTMount_{firstDayObs}-{lastDayObs}.pdf"))
fig = plt.figure(figsize=(10, 8))

for i, [dayObs, blockNum] in enumerate(dayBlockPairs):
    events = eventMaker.getEvents(dayObs)
    these_events = []
    for e in events:
        try:
            for b in e.blockInfos:
                if b.blockNumber==blockNum:
                    these_events.append(e)
                    break
        except:
            continue
    tracks = [e for e in these_events if e.type.name=="TRACKING"]
    for track in tracks:
        try:
            # Get the information from the previous slew
            try:
                seqNum = track.seqNum
                previous_slew = getPreviousSlew(these_events, seqNum)
                if previous_slew.type.name == 'SLEWING':
                    azimuthData, elevationData = getAzimuthElevationDataForEvent(
                        client, previous_slew)
                    azShift = abs(azimuthData['actualPosition'].iloc[0]
                                  - azimuthData['actualPosition'].iloc[-1])
                    elShift = abs(elevationData['actualPosition'].iloc[0]
                                  - elevationData['actualPosition'].iloc[-1])
                    azShiftMod = azShift * np.cos(elevationData['actualPosition'].iloc[0]*np.pi/180.0)
                    shift = np.sqrt(elShift*elShift + azShiftMod*azShiftMod)
                    if shift < 0.2 or shift > 10.0:
                        continue
                else:
                    # If the previous event wasn't a slew, we won't use this track.
                    print(f"Discarding {dayObs}-{seqNum} because it isn't preceded by a slew")
                    continue
            except:
                print(f"Discarding {dayObs}-{seqNum} because of an error processing the slew")
                continue
    
            # Now get the track information
            azimuthData, elevationData = getAzimuthElevationDataForEvent(
                    client, track, postPadding = -1.0)
            thisEl = elevationData['actualPosition'].iloc[0]
            azError = azimuthData['azError'].values
            elError = elevationData['elError'].values
            azBadValues = filterBadValues(azError)
            elBadValues = filterBadValues(elError)
            azimuthData['azError'] = azError
            elevationData['elError'] = elError
            azError = azimuthData['azError']
            elError = elevationData['elError']

            initialAzRms = np.sqrt(np.mean(azError**2))
            initialElRms = np.sqrt(np.mean(elError**2))
            if initialAzRms > 10.0 or initialElRms > 10.0:
                print(f"Discarding {dayObs}-{seqNum} because the slew is out of limits")
                continue
    
            
            # Now calculate the settling time
            timeToSettle = binary_search(track, azimuthData, elevationData, thisEl, 
                  0.0, 10.0, precision=0.1)
            timeSettled = track.begin + TimeDelta(timeToSettle, format='sec')
            timeToSettles.append(timeToSettle)
            slewTimesSettled.append(previous_slew.duration + timeToSettle)

            # Now calculate the RMS error from the inPosition time
            # plus an offset and for 15 seconds afterward
            for delay in delays:
                rmsError = jitterNextFifteen(track, azimuthData, 
                                             elevationData, thisEl, delay)
                rmsErrors[delay].append(rmsError)

            # Save the slewing information
            slewDistance.append(shift)
            slewTimesInPosition.append(previous_slew.duration)
            azVals.append(azimuthData['actualPosition'].iloc[0])
            elVals.append(thisEl)
            rmsError0s.append(max(azError.iloc[0], elError.iloc[0]))
            
            # Now make the plots
            plotEvent(client, track, fig, azimuthData=azimuthData, 
              elevationData=elevationData)
            ax = fig.get_axes()[1]
            ax.axvline(track.begin.utc.datetime, ls='--', color='black')
            ax.axvline(timeSettled.utc.datetime, ls='--', color='green')
            ax.text(0.1, 0.8,
                   f'{azBadValues} bad azimuth values and {elBadValues} bad elevation values were replaced',
                       transform=ax.transAxes)
            print(f"Event {dayObs}-{seqNum} was a success")
            pdf.savefig(fig)  # saves the current figure into a pdf page
            plt.clf()
        except:
            print("Unkown failure")
            continue
pdf.close()

# Now plot the slew / settle time histograms.

In [ ]:
%matplotlib inline
timeToSettles = np.array(timeToSettles)
goodSettles = timeToSettles[timeToSettles < 0.000001]
percentGoodSettles = len(goodSettles) / len(timeToSettles) * 100.0

fig = plt.figure(figsize=(10, 5))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"MT Mount Time for jitter to settle\n{firstDayObs}-{lastDayObs}", fontsize = 18, y=1.05)
plt.subplot(1,1,1)
plt.title(f"N = {len(timeToSettles)}, {percentGoodSettles:.1f} % are settled at InPosition")
plt.hist(timeToSettles, bins=100, range=(0,10))
plt.xlim(0,10)
plt.xlabel("Time to settle (seconds)")
plt.savefig(str(dataDir / f"Settling_Time_{firstDayObs}-{lastDayObs}.png"))

In [ ]:
%matplotlib inline
slewsLessThan3p5 = 0
slewsMeetingSpec = 0
for i in range(len(slewTimesInPosition)):
    if slewDistance[i] <= 3.5:
        slewsLessThan3p5 += 1
        if slewTimesInPosition[i] < 4.0:
            slewsMeetingSpec += 1
percentInSpec = slewsMeetingSpec / slewsLessThan3p5 *100.0
fig = plt.figure(figsize=(10, 5))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"MT Mount Time from Slew Start to InPosition\n{firstDayObs}-{lastDayObs}", fontsize = 18, y=1.05)
plt.subplot(1,2,1)
plt.hist(slewTimesInPosition, bins=50, range=(0,50))
plt.xlim(0,10)
plt.xlabel("Slew and settle time (seconds)")
plt.subplot(1,2,2)
plt.scatter(slewDistance, slewTimesInPosition)
plt.ylabel("Slew and settle time(sec)")
plt.xlabel("Slew distance (degrees)")
plt.plot([3.5,3.5],[0,10], ls='--', color='black')
plt.plot([0,10],[4.0,4.0], ls='--', color='black')
plt.text(4.0, 8.0,f"{percentInSpec:.1f} percent of the slews \nless than 3.5 degrees take \nless than 4 seconds")
plt.xlim(0,10)
plt.ylim(0,10)
plt.savefig(str(dataDir / f"Slew_Settle_Times_InPosition_{firstDayObs}-{lastDayObs}.png"))

In [ ]:
%matplotlib inline
slewsLessThan3p5 = 0
slewsMeetingSpec = 0
for i in range(len(slewTimesSettled)):
    if slewDistance[i] <= 3.5:
        slewsLessThan3p5 += 1
        if slewTimesSettled[i] < 4.0:
            slewsMeetingSpec += 1
percentInSpec = slewsMeetingSpec / slewsLessThan3p5 *100.0

fig = plt.figure(figsize=(10, 5))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"MT Mount Time from Slew Start to Jitter Settled\n{firstDayObs}-{lastDayObs}", fontsize = 18, y=1.05)
plt.subplot(1,2,1)
plt.hist(slewTimesSettled, bins=50, range=(0,50))
plt.xlim(0,20)
plt.xlabel("Slew and settle time (seconds)")
plt.subplot(1,2,2)
plt.scatter(slewDistance, slewTimesSettled)
plt.ylabel("Slew and settle time(sec)")
plt.xlabel("Slew distance (degrees)")
plt.plot([3.5,3.5],[0,10], ls='--', color='black')
plt.plot([0,10],[4.0,4.0], ls='--', color='black')
plt.text(4.0, 8.0,f"{percentInSpec:.1f} percent of the slews \nless than 3.5 degrees take \nless than 4 seconds")
plt.xlim(0,10)
plt.ylim(0,10)
plt.savefig(str(dataDir / f"Slew_Settle_Times_Settled_{firstDayObs}-{lastDayObs}.png"))

# This plots histograms of the tracking jitter.

In [ ]:
%matplotlib inline
fig, azs = plt.subplots(2,2,figsize=(10,10))
plt.suptitle(f"MT Mount Error histograms - {firstDayObs}-{lastDayObs}", fontsize = 18)#, y=1.05)
plt.subplots_adjust(hspace=0.5)
for i, delay in enumerate(delays):
    azx = i % 2
    azy = int(i / 2)
    rmsError = rmsErrors[delay]
    rmsError = np.array(rmsError)
    goodRms = rmsError[rmsError < 0.01]
    percentGoodRms = len(goodRms) / len(rmsError) * 100.0
    rmsMed = np.nanmedian(rmsError)
    azs[azy][azx].set_title(f"From InPosition + {delay:.0f} sec, ending 15 sec later.\nN={len(rmsError)}, Median={rmsMed:.3f}, Less than 0.01 = {percentGoodRms:.1f}")
    azs[azy][azx].hist(rmsError, range=(0,0.1), bins=100)
    azs[azy][azx].axvline(0.01, color='black', ls = '--')
    azs[azy][azx].set_xlim(0,0.05)
    azs[azy][azx].set_xlabel("RMS Jitter (arcseconds)")
plt.savefig(str(dataDir / f"Jitter_Summary_{firstDayObs}-{lastDayObs}.png"))

# This plots histograms of the tracking error at the beginning of the time period.

In [ ]:
%matplotlib inline
rmsError0s = np.array(rmsError0s)
goodErrors = rmsError0s[rmsError0s < 0.1]
percentGoodErrors = len(goodErrors) / len(rmsError0s) * 100.0
fig = plt.figure(figsize=(10,5))
plt.suptitle(f"MT Mount Error at inPosition - {firstDayObs}-{lastDayObs}", fontsize = 18, y=1.05)
rmsMed = np.nanmedian(rmsError0s)
plt.subplot(1,1,1)
plt.title(f"Worst case tracking error at inPosition\n N={len(rmsError0s)} Median = {rmsMed:.3f}  Less than 0.1 = {percentGoodErrors:.1f} %")
plt.hist(rmsError0s, range=(0,0.5), bins=50)
plt.axvline(0.1, color='black', ls = '--')
plt.xlim(0,0.5)
plt.xlabel("Tracking error (arcseconds)")
plt.savefig(str(dataDir / f"InPosition_Error_{firstDayObs}-{lastDayObs}.png"))